In [41]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
from utility import Utility as util

In [43]:
import pandas as pd
pd.options.display.max_columns = None
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
import warnings
import time
import os
#import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')
#from libraries.project_reports import project_reports
#from libraries.utility import Utility as util
mutil = util()
#print(img_text_data)

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))

img_path = os.path.join(mutil.get_this_dir(),"artifacts","images","irwin_analytics_small.txt")
img_text_data = mutil.get_data_from_file(img_path)

img_tag = "<img align=\"left\" src=\"data:image/png;base64,{}\" />".format(img_text_data)
a_tag = "<a href=\"https://www.irwinanalytics.com\" target=\"https://www.irwinanalytics.com\">{}</a>".format(img_tag)

my_html = "<hr/>".format(img_tag,a_tag)
display(HTML(my_html))
my_html = "<div class='parent'><div>{}</div></div>".format(a_tag)
display(HTML(my_html))

## System Overview -- The Environment

Our streaming and transfomration solution is virtualized in this case.   

We create this utilizing a [yaml](docker-compose.yml,yml) file under the docker-compose framework.




In [44]:
img_path = os.path.join(mutil.get_this_dir(),"artifacts","images","docker_network.png")

img_tag = mutil.get_embedded_image_tag_from_image_file(img_path)

my_html = "<hr/>{}".format(img_tag)
display(HTML(my_html))


## System Overview -- The data flow

Below is a informal representation of the application's data flow.

In [45]:
img_path = os.path.join(mutil.get_this_dir(),"artifacts","images","streaming_visualization.jpg")

img_tag = mutil.get_embedded_image_tag_from_image_file(img_path)

my_html = "<hr/>{}".format(img_tag)
display(HTML(my_html))

## Using Presto to Generate Reports

To generate reports for those interested in application usage, we can use Presto via the PyHive connector to load our Hive tables into Pandas dataframes, transform the dataframes to answer our business questions, and then publish the reports as JSON files.

Before we start, let's define two business questions that we would like to answer:

1. What are all the counts per event type?
2. What are all the parameters that were given for the `user` parameter?

First, let's install the PyHive library.

In [46]:
%%capture output
!python3 -m pip install git+https://github.com/dropbox/PyHive.git >/dev/null

Next, let's use PyHive to connect to Presto in code using the port that we exposed in our Docker Compose file. Once we connect, we can run a simple query to see all the tables that are created in Hive.

In [47]:
from pyhive import presto
import pandas as pd

presto_conn = presto.connect(
    host='presto',
    port=8080 # Exposed Presto port (see docker compose file)
)

pd.read_sql_query("SHOW TABLES", presto_conn)

,Table
0,all_events
1,event_parameters


Now let's run a query to get all of the data from the `event_parameters` table and load it into a Pandas dataframe.

In [48]:
# https://stackoverflow.com/questions/55988436/how-to-convert-a-presto-query-output-to-a-python-data-frame
event_parameters = pd.read_sql_query("SELECT * from event_parameters", presto_conn)
event_parameters.tail()

,raw_event,timestamp,accept,host,user_agent,event_id,parameter_name,parameter_value
158551,"{""event_id"": ""05bff67c-1604-410b-a3f5-4cb716bb...",2023-08-30 12:46:19.725,None,None,None,05bff67c-1604-410b-a3f5-4cb716bb43a1,user,aastha
158552,"{""event_id"": ""682b2fdf-814f-4e60-a4b7-e3770d8e...",2023-08-30 12:46:19.726,None,None,None,682b2fdf-814f-4e60-a4b7-e3770d8e7926,sword_type,Heartseeker
158553,"{""event_id"": ""682b2fdf-814f-4e60-a4b7-e3770d8e...",2023-08-30 12:46:19.727,None,None,None,682b2fdf-814f-4e60-a4b7-e3770d8e7926,user,aastha
158554,"{""event_id"": ""543222b3-ded0-40f1-9627-85a2e294...",2023-08-30 12:46:19.728,None,None,None,543222b3-ded0-40f1-9627-85a2e294e7ce,sword_type,Heartseeker
158555,"{""event_id"": ""543222b3-ded0-40f1-9627-85a2e294...",2023-08-30 12:46:19.728,None,None,None,543222b3-ded0-40f1-9627-85a2e294e7ce,user,aastha


Let's do the same thing for the data in the `all_events` table.

In [49]:
all_events = pd.read_sql_query("SELECT * from all_events", presto_conn)
all_events.tail()

,raw_event,timestamp,accept,host,user_agent,event_id,event_type
79273,"{""event_id"": ""228ca62e-bc99-44a1-94e2-27c71242...",2023-08-30 12:59:49.96,*/*,user2.att.com,ApacheBench/2.3,228ca62e-bc99-44a1-94e2-27c71242fec0,get_credit
79274,"{""event_id"": ""5a847eb6-d42d-4c98-bdb8-ab05cba1...",2023-08-30 12:59:49.962,*/*,user2.att.com,ApacheBench/2.3,5a847eb6-d42d-4c98-bdb8-ab05cba12cca,get_credit
79275,"{""event_id"": ""41ea15cd-196d-48ee-8151-ed55f993...",2023-08-30 12:59:49.963,*/*,user2.att.com,ApacheBench/2.3,41ea15cd-196d-48ee-8151-ed55f993510d,get_credit
79276,"{""event_id"": ""9910228b-6269-410d-b13d-7ad65960...",2023-08-30 12:59:49.965,*/*,user2.att.com,ApacheBench/2.3,9910228b-6269-410d-b13d-7ad6596040dc,get_credit
79277,"{""event_id"": ""88affd35-1339-465a-8882-3445341e...",2023-08-30 12:59:49.966,*/*,user2.att.com,ApacheBench/2.3,88affd35-1339-465a-8882-3445341efe6b,get_credit


Now let's try to answer business question #1 - What are all the counts per event type? We can do this using a simple groupby statement on our `all_events` dataframe and then writing the output to a JSON file titled `event_type_count.json`.

In [50]:
event_type_count = all_events.groupby('event_type').size()
event_type_count.to_json("event_type_count.json", orient='columns')

Note that `event_type_count.json` should exist in your directory after running the code above.


In [51]:
query_data = mutil.get_data_from_file(os.path.join(mutil.get_this_dir(),"hive_queries","all_events.sql"))

all_events = pd.read_sql_query(query_data, presto_conn)
all_events.head()

,event_type,event_count
0,purchase_sword,20764
1,leave_guild,20402
2,join_guild,19103
3,get_credit,19009




Now let's answer business question #2 - What are all the parameters that were given for the `user` parameter? We can do this by running a slightly more complex query on our dataframe and again writing the output to a JSON file.

In [52]:
# Question: What are all the parameters that were given for the `user` parameter?
user_parameter_count = event_parameters.where(event_parameters['parameter_name'] == 'user').groupby('parameter_value').size()
user_parameter_count.to_json("user_parameter_count.json", orient='columns')

query_data = mutil.get_data_from_file(os.path.join(mutil.get_this_dir(),"hive_queries","get_user_event_counts.hsql"))

events_by_user = pd.read_sql_query(query_data, presto_conn)

events_by_user.head(50)


,user,event,user_event_count
0,aastha,get_credit,3318
1,ben,get_credit,4416
2,don,get_credit,3776
3,lise,get_credit,3496
4,theresa,get_credit,4003
5,aastha,join_guild,3830
6,ben,join_guild,3660
7,don,join_guild,3730
8,lise,join_guild,3935
9,theresa,join_guild,3948


In [79]:
query_dir_dict = mutil.load_subdirs_into_dict(os.path.join(mutil.get_this_dir(),"hive_queries","report_queries"),"sql")

rep_for_display = dict()

for qd in query_dir_dict["report_queries"]:
    for key in qd.keys():
        if ".sql_content" in key:
            name = key.replace(".sql_content","")
            #if "GET EVENT TOTALS" in name:
            display(HTML(name))
            report_payload = pd.read_sql_query(qd[key], presto_conn)
            display(report_payload)
            rep_for_display[name]=report_payload
        

,user_name,event,guild_leave_count
0,lise,leave_guild,4246
1,don,leave_guild,4190
2,aastha,leave_guild,4092
3,ben,leave_guild,4063
4,theresa,leave_guild,3811


,user_name,event,guid_join_count
0,theresa,join_guild,3948
1,lise,join_guild,3935
2,aastha,join_guild,3830
3,don,join_guild,3730
4,ben,join_guild,3660


,sword_name,event,popular_sword_count
0,The Fury,purchase_sword,350
1,Time Raider,purchase_sword,323
2,Orcbane,purchase_sword,317
3,Ice Finger,purchase_sword,303
4,Sky striker,purchase_sword,295
5,Diplomacy,purchase_sword,283
6,Stromen Richter,purchase_sword,262
7,Dawn Reach,purchase_sword,261
8,Skull Splitter,purchase_sword,259
9,Elysium,purchase_sword,258


,event_type,event_count
0,purchase_sword,20764
1,leave_guild,20402
2,join_guild,19103
3,get_credit,19009


,guild_name,event,popular_guild_count
0,Paladins of Claws,join_guild,1621
1,Rising Vitality,join_guild,1699
2,Humble Force,join_guild,1837
3,Fortune of the Bear,join_guild,1884
4,Host of the Natural,join_guild,1920
5,Bright Hearts,join_guild,1960
6,Knights of the Scorpion,join_guild,1981
7,World Paradox,join_guild,1995
8,Midnightfury,join_guild,2075
9,Banner Foxes,join_guild,2131


,user_name,event,get_credit_count
0,ben,get_credit,4416
1,theresa,get_credit,4003
2,don,get_credit,3776
3,lise,get_credit,3496
4,aastha,get_credit,3318


Now that we have answered the business questions and generated JSON reports with the answers, we can close our Presto connection.

In [54]:
presto_conn.close()